<a href="https://colab.research.google.com/github/amit-raj-repo/CNN-Malaria-Detection-Using-Keras/blob/master/Malaria_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import os
import zipfile
import random 
from shutil import copyfile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Unzip the data
local_zip = '/content/gdrive/My Drive/Malaria-Detection/cell_images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/cell-images-for-detecting-malaria')
zip_ref.close()

In [0]:
#Getting image lists for 2 different classes
parapath = '/content/cell-images-for-detecting-malaria/cell_images/Parasitized/'
uninpath = '/content/cell-images-for-detecting-malaria/cell_images/Uninfected/'
parastized = os.listdir(parapath)
uninfected = os.listdir(uninpath)

In [5]:
len(parastized)

13780

In [6]:
data = []
label = []


for para in parastized:
    try:
      img =  image.load_img(parapath+para,target_size=(64,64))
      x = image.img_to_array(img)
      data.append(x)
      label.append(1)
    except:
        print("Can't add "+para+" in the dataset")
        
for unin in uninfected:
    try:
        img =  image.load_img(uninpath+unin,target_size=(64,64))
        x = image.img_to_array(img)
        data.append(x)
        label.append(0)
    except:
        print("Can't add "+unin+" in the dataset")

Can't add Thumbs.db in the dataset
Can't add Thumbs.db in the dataset


In [0]:
#Changing the image data to array and divide by 255 to restrict the data range to 0-1
data = np.array(data)/255
label = np.array(label)

In [16]:
print(data.shape)

(27558, 64, 64, 3)


In [0]:
#Saving the datasets for future use
np.save("/content/gdrive/My Drive/Malaria-Detection/malaria_image_data_64.npy", data)
np.save("/content/gdrive/My Drive/Malaria-Detection/malaria_image_label_64.npy", label)

In [0]:
#==========================================================================#
# Model Building
#==========================================================================#

In [0]:
#Loading the data
imageData = np.load('/content/gdrive/My Drive/Malaria-Detection/malaria_image_data_64.npy')
imageLabels = np.load('/content/gdrive/My Drive/Malaria-Detection/malaria_image_label_64.npy')

In [0]:
#Creating Test and train Datasets

X_train, X_test, y_train, y_test = train_test_split(imageData,
                                                    imageLabels,
                                                    test_size=0.10,
                                                    random_state=42)

In [21]:
#User Defined Model

cnnModel = Sequential()

cnnModel.add(Conv2D(32, (3,3), (1,1), activation= 'relu', input_shape = (64,64,3)))
cnnModel.add(Conv2D(32, (3,3), (1,1), activation= 'relu'))
cnnModel.add(MaxPool2D(pool_size= (2,2)))


cnnModel.add(Conv2D(64, (3,3), (1,1), activation= 'relu'))
cnnModel.add(Dropout(0.1, seed=42))

cnnModel.add(Conv2D(64, (3,3), (1,1), activation= 'relu'))
cnnModel.add(MaxPool2D(pool_size= (2,2)))

cnnModel.add(Conv2D(128, (3,3), (1,1), activation= 'relu'))
cnnModel.add(Dropout(0.1, seed=42))

cnnModel.add(Conv2D(128, (3,3), (1,1), activation= 'relu'))
cnnModel.add(MaxPool2D(pool_size= (2,2)))

cnnModel.add(Flatten())
cnnModel.add(Dense(128, activation= 'relu'))
cnnModel.add(Dropout(0.1, seed=42))

cnnModel.add(Dense(1, activation= 'sigmoid'))

cnnModel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
dropout_4 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 26, 26, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 64)       

In [0]:
adam = Adam(learning_rate= 0.001, beta_1 = 0.9, beta_2 = 0.99)

In [0]:
cnnModel.compile(optimizer=adam, loss= 'binary_crossentropy', metrics= ['accuracy'])

In [24]:
cnnModel.fit(X_train,y_train, validation_data=(X_test, y_test), batch_size= 300, epochs= 5, verbose= 1)

Epoch 1/5
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4020 - accuracy: 0.7840 - val_loss: 0.1537 - val_accuracy: 0.9557
Epoch 2/5
1000/1000 [==============================] - 23s 23ms/step - loss: 0.1622 - accuracy: 0.9504 - val_loss: 0.1361 - val_accuracy: 0.9601
Epoch 3/5
1000/1000 [==============================] - 23s 23ms/step - loss: 0.1462 - accuracy: 0.9546 - val_loss: 0.1361 - val_accuracy: 0.9590
Epoch 4/5
1000/1000 [==============================] - 23s 23ms/step - loss: 0.1415 - accuracy: 0.9547 - val_loss: 0.1363 - val_accuracy: 0.9586
Epoch 5/5
 965/1000 [===========================>..] - 22s 23ms/step - loss: 0.1363 - accuracy: 0.9566 - val_loss: 0.1233 - val_accuracy: 0.9597
